# **INTRODUCTIONS**

---

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **DISABILITY_CARE - NL**

---

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[DISABILITY_ARCHIVED_20230126]'))
CREATE TABLE [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126] (
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[INFECTED_DISABILITY_DAILY] [int] NULL,
	[DECEASED_DISABILITY_DAILY] [int] NULL,
	[TOTAL_NEW_INFECTED_LOCATIONS] [int] NULL,
	[TOTAL_INFECTED_LOCATIONS] [int] NULL,
	[TOTAL_LOCATIONS] [int] NULL,
	[PERCENTAGE_INFECTED_LOCATIONS] [float] NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_RANGE_START] [datetime] NULL,
	[DATE_OF_REPORTS_LAG] [datetime] NULL,
	[DATE_RANGE_START_LAG] [datetime] NULL,
	[7D_AVERAGE_TOTAL_INFECTED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_INFECTED_LAG] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED_LAG] [decimal](16, 2) NULL
);
GO


Stored procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_DISABILITY_ARCHIVED_20230126] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126]

    INSERT INTO [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126] (
        	[ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [INFECTED_DISABILITY_DAILY],
            [DECEASED_DISABILITY_DAILY],
            [TOTAL_NEW_INFECTED_LOCATIONS],
            [TOTAL_INFECTED_LOCATIONS],
            [TOTAL_LOCATIONS],
            [PERCENTAGE_INFECTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    )
    SELECT 
            [ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [INFECTED_DISABILITY_DAILY],
            [DECEASED_DISABILITY_DAILY],
            [TOTAL_NEW_INFECTED_LOCATIONS],
            [TOTAL_INFECTED_LOCATIONS],
            [TOTAL_LOCATIONS],
            [PERCENTAGE_INFECTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    FROM [VWSDEST].[DISABILITY]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[DISABILITY])
END


View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DISABILITY_NATIONAL_ARCHIVED_20230126]
AS
    SELECT
    [INFECTED_DISABILITY_DAILY] AS newly_infected_people,
    [7D_AVERAGE_TOTAL_INFECTED] AS NEWLY_INFECTED_PEOPLE_MOVING_AVERAGE,
    [TOTAL_NEW_INFECTED_LOCATIONS] AS newly_infected_locations,
    [TOTAL_INFECTED_LOCATIONS] AS infected_locations_total,
    [PERCENTAGE_INFECTED_LOCATIONS] AS infected_locations_percentage,
    [DECEASED_DISABILITY_DAILY] AS deceased_daily,
    [7D_AVERAGE_TOTAL_DECEASED] AS DECEASED_DAILY_MOVING_AVERAGE,
    [DATE_OF_REPORT_UNIX] AS date_unix,
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS date_of_insertion_unix
    FROM [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126]
    WHERE DATE_OF_REPORT >=  '2020-02-27 00:00:00.000'
    AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126])
    AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[DISABILITY_ARCHIVED_20230126])
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
@item_name VARCHAR(256) = 'disability_care',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DISABILITY_NATIONAL_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'disability_care_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

# **DISABILITY_CARE - VR**

---

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126]'))
CREATE TABLE [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126](
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[VRCODE] [varchar](100) NULL,
	[INFECTED_DISABILITY_DAILY] [int] NULL,
	[DECEASED_DISABILITY_DAILY] [int] NULL,
	[TOTAL_NEW_INFECTED_LOCATIONS] [int] NULL,
	[TOTAL_INFECTED_LOCATIONS] [int] NULL,
	[TOTAL_LOCATIONS] [int] NULL,
	[PERCENTAGE_INFECTED_LOCATIONS] [float] NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_RANGE_START] [datetime] NULL,
	[DATE_OF_REPORTS_LAG] [datetime] NULL,
	[DATE_RANGE_START_LAG] [datetime] NULL,
	[7D_AVERAGE_TOTAL_INFECTED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_INFECTED_LAG] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED_LAG] [decimal](16, 2) NULL
);
GO


Stored Procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_DISABILITY_VR_ARCHIVED_20230126] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126]

    INSERT INTO [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126] (
        	[ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [VRCODE],
            [INFECTED_DISABILITY_DAILY],
            [DECEASED_DISABILITY_DAILY],
            [TOTAL_NEW_INFECTED_LOCATIONS],
            [TOTAL_INFECTED_LOCATIONS],
            [TOTAL_LOCATIONS],
            [PERCENTAGE_INFECTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    )
    SELECT 
        	[ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [VRCODE],
            [INFECTED_DISABILITY_DAILY],
            [DECEASED_DISABILITY_DAILY],
            [TOTAL_NEW_INFECTED_LOCATIONS],
            [TOTAL_INFECTED_LOCATIONS],
            [TOTAL_LOCATIONS],
            [PERCENTAGE_INFECTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    FROM [VWSDEST].[DISABILITY_VR]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[DISABILITY_VR])
END


View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DISABILITY_VR_ARCHIVED_20230126]
AS
    SELECT
    [DATE_OF_REPORT_UNIX] AS date_unix,
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS date_of_insertion_unix,
    [VRCODE] as vrcode,
    [INFECTED_DISABILITY_DAILY] AS newly_infected_people,
    [7D_AVERAGE_TOTAL_INFECTED] AS NEWLY_INFECTED_PEOPLE_MOVING_AVERAGE,
    [TOTAL_NEW_INFECTED_LOCATIONS] AS newly_infected_locations,
    [TOTAL_INFECTED_LOCATIONS] AS infected_locations_total,
    [PERCENTAGE_INFECTED_LOCATIONS] AS infected_locations_percentage,
    [DECEASED_DISABILITY_DAILY] AS deceased_daily,
    [7D_AVERAGE_TOTAL_DECEASED] AS DECEASED_DAILY_MOViNG_AVERAGE
    FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126] WITH(NOLOCK)
    WHERE DATE_OF_REPORT >=  '2020-02-27 00:00:00.000'
    AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126] WITH(NOLOCK))
    AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126] WITH(NOLOCK))
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
@item_name VARCHAR(256) = 'disability_care',
@proto_name VARCHAR(50) = 'VR%',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DISABILITY_VR_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'disability_care_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(1024) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

# **DISABILITY_CARE - VR_COLLECTION**

---

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DISABILITY_REGIONS_ARCHIVED_20230126]
AS
    SELECT
    [INFECTED_DISABILITY_DAILY] AS newly_infected_people,
    [TOTAL_NEW_INFECTED_LOCATIONS] AS newly_infected_locations,
    [TOTAL_INFECTED_LOCATIONS] AS infected_locations_total,
    [PERCENTAGE_INFECTED_LOCATIONS] AS infected_locations_percentage,
    [DECEASED_DISABILITY_DAILY] AS deceased_daily,
    [DATE_OF_REPORT_UNIX] AS date_unix,
    dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS date_of_insertion_unix,
    [VRCODE] as vrcode
    FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126]
    WHERE DATE_OF_REPORT >=  '2020-02-27 00:00:00.000'
    --For the regions file, we only need the T-1 date_of_report record per VR
    AND DATE_OF_REPORT = (SELECT DATE_OF_REPORT from 
    (SELECT DATE_OF_REPORT, RANK() OVER (ORDER by date_of_report desc) ranking
    FROM (SELECT DISTINCT DATE_OF_REPORT FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126])) x) y
    WHERE ranking = 2)
    AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[DISABILITY_VR_ARCHIVED_20230126])
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
@item_name VARCHAR(256) = 'disability_care',
@proto_name VARCHAR(50) = 'VR_COLLECTION',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DISABILITY_REGIONS_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'disability_care_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'VR_COLLECTION',
        @columns VARCHAR(1024) = '*',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO